In [158]:
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [159]:
df=pd.read_csv("Marvel_Comics.csv")
df.head()

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price
0,A Year of Marvels: April Infinite Comic (2016),(2016),A Year of Marvels: April Infinite Comic (2016) #1,"April 01, 2016",The Infinite Comic that will have everyone tal...,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,Free
1,A Year of Marvels: August Infinite Comic (2016),(2016),A Year of Marvels: August Infinite Comic (2016...,"August 10, 2016","It’s August, and Nick Fury is just in time to ...",Jamal Campbell,"Chris Sims, Chad Bowers",NaN,Marvel Universe,Infinite Comic,NaN,Free
2,A Year of Marvels: February Infinite Comic (2016),(2016),A Year of Marvels: February Infinite Comic (20...,"February 10, 2016",Join us in a brand new Marvel comics adventure...,"Danilo S. Beyruth, M Mast",Ryan North,NaN,Marvel Universe,Infinite Comic,Rated T+,Free
3,A Year of Marvels: July Infinite Comic (2016),(2016),A Year of Marvels: July Infinite Comic (2016) #1,"June 29, 2016",Celebrating the Fourth of July is complicated ...,Juanan Ramirez,Chuck Wendig,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free
4,A Year of Marvels: June Infinite Comic (2016),(2016),A Year of Marvels: June Infinite Comic (2016) #1,"June 15, 2016",Sam Alexander’s finding it hard to cope with t...,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,Free


In [160]:
df.shape

(34992, 12)

In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34992 entries, 0 to 34991
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   comic_name         34992 non-null  object
 1   active_years       34992 non-null  object
 2   issue_title        34466 non-null  object
 3   publish_date       34466 non-null  object
 4   issue_description  30395 non-null  object
 5   penciler           25482 non-null  object
 6   writer             27595 non-null  object
 7   cover_artist       12255 non-null  object
 8   Imprint            11684 non-null  object
 9   Format             32894 non-null  object
 10  Rating             12619 non-null  object
 11  Price              32894 non-null  object
dtypes: object(12)
memory usage: 3.2+ MB


In [162]:
df.describe()

,comic_name,active_years,issue_title,publish_date,issue_description,penciler,writer,cover_artist,Imprint,Format,Rating,Price
count,34992,34992,34466,34466,30395,25482,27595,12255,11684,32894,12619,32894
unique,4935,412,33757,3317,29331,3915,3082,1010,39,11,36,53
top,Uncanny X-Men (1981 - 2011),(1968 - 1996),X-Men: The Complete Age of Apocalypse Epic Boo...,"November 30, -0001",Please note that these digital editions collec...,Sal Buscema,Brian Michael Bendis,Gil Kane,Marvel Universe,Comic,Rated T+,Free
freq,588,718,10,543,37,394,859,217,7786,31899,3204,15136


In [163]:
client = MongoClient("mongodb://localhost:27017")
db = client['marvel']

collections = ["raw_marvel", "curated_marvel", "analytics_marvel"]

for coll in collections:
    if coll not in db.list_collection_names():
        db.create_collection(coll)
        print(f"Colección '{coll}' creada")
    else:
        print(f"Colección '{coll}' ya existe")



Colección 'raw_marvel' ya existe
Colección 'curated_marvel' ya existe
Colección 'analytics_marvel' ya existe


In [164]:
col = db["raw_marvel"] 



In [165]:
df_raw=df.copy()

In [166]:
df_raw.drop(columns=["issue_description","issue_title"],inplace=True)

In [167]:
#NORMALIZACIÓN DE NOMBRES DE COLUMNAS
cols=df_raw.columns.astype(str).str.title()
print(cols)

Index(['Comic_Name', 'Active_Years', 'Publish_Date', 'Penciler', 'Writer',
       'Cover_Artist', 'Imprint', 'Format', 'Rating', 'Price'],
      dtype='object')


In [168]:
#EXTRAEMOS LOS AÑOS DE LOS PARÉNTESIS Y LOS CONVERTIMOS A ENTEROS
df_raw["active_years"] = df_raw["active_years"].str.extract(r"\((\d{4})")[0].astype("int64")

In [169]:
#CAMBIAMOS LOS PRECIOS A FLOAT
df_raw["Price"] = df_raw["Price"].str.strip().replace("Free", "0")  # Free → 0
df_raw["Price"] = df_raw["Price"].str.replace(r"[\$,]", "", regex=True)  # quitar $ y comas
df_raw["Price"] = df_raw["Price"].astype("float64")

In [170]:
#CAMBIAMOS EL FORMATO DE FECHA
df_raw["publish_date"] = pd.to_datetime(df_raw["publish_date"], errors="coerce")

In [171]:
df_raw.head()

,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price
0,A Year of Marvels: April Infinite Comic (2016),2016,2016-04-01,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,0.0
1,A Year of Marvels: August Infinite Comic (2016),2016,2016-08-10,Jamal Campbell,"Chris Sims, Chad Bowers",NaN,Marvel Universe,Infinite Comic,NaN,0.0
2,A Year of Marvels: February Infinite Comic (2016),2016,2016-02-10,"Danilo S. Beyruth, M Mast",Ryan North,NaN,Marvel Universe,Infinite Comic,Rated T+,0.0
3,A Year of Marvels: July Infinite Comic (2016),2016,2016-06-29,Juanan Ramirez,Chuck Wendig,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,0.0
4,A Year of Marvels: June Infinite Comic (2016),2016,2016-06-15,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,NaN,0.0


In [172]:
#CAMBIAMOS LOS NAN DE STRING A NULL
obj_cols = df_raw.select_dtypes(include="O").columns

df_raw[obj_cols] = df_raw[obj_cols].where(df_raw[obj_cols].notna(), None)

In [173]:
df_raw.head()

,comic_name,active_years,publish_date,penciler,writer,cover_artist,Imprint,Format,Rating,Price
0,A Year of Marvels: April Infinite Comic (2016),2016,2016-04-01,Yves Bigerel,Yves Bigerel,Jamal Campbell,Marvel Universe,Infinite Comic,Rated T+,0.0
1,A Year of Marvels: August Infinite Comic (2016),2016,2016-08-10,Jamal Campbell,"Chris Sims, Chad Bowers",None,Marvel Universe,Infinite Comic,None,0.0
2,A Year of Marvels: February Infinite Comic (2016),2016,2016-02-10,"Danilo S. Beyruth, M Mast",Ryan North,None,Marvel Universe,Infinite Comic,Rated T+,0.0
3,A Year of Marvels: July Infinite Comic (2016),2016,2016-06-29,Juanan Ramirez,Chuck Wendig,Jamal Campbell,Marvel Universe,Infinite Comic,None,0.0
4,A Year of Marvels: June Infinite Comic (2016),2016,2016-06-15,Diego Olortegui,Paul Allor,Jamal Campbell,Marvel Universe,Infinite Comic,None,0.0


In [174]:
print(df_raw.isna().sum())

comic_name          0
active_years        0
publish_date     1069
penciler         9510
writer           7397
cover_artist    22737
Imprint         23308
Format           2098
Rating          22373
Price            2098
dtype: int64


In [175]:
print(df_raw.isna().sum()/len(df_raw))

comic_name      0.000000
active_years    0.000000
publish_date    0.030550
penciler        0.271776
writer          0.211391
cover_artist    0.649777
Imprint         0.666095
Format          0.059957
Rating          0.639375
Price           0.059957
dtype: float64


In [176]:
#VAMOS A COMPROBAR QUE CATEGORIAS DE RATING HAY
unique_values = df_raw["Rating"].unique()
print(unique_values)



[' Rated T+' None ' Rated T' ' ALL AGES' ' A' ' Parental Advisory'
 ' Marvel Psr' ' No Rating' ' MARVEL PSR' ' T' ' RATED T' ' Max'
 ' RATED T+' ' RATED A' ' All Ages' ' T+' ' Rated a' ' Rated A'
 ' Parental Advisory/Explicit Content' ' PARENTAL SUPERVISION'
 ' PARENTAL ADVISORY' ' Mature' ' MARVEL PSR+' ' EXPLICIT CONTENT'
 ' PARENTAL ADVISORYSLC' ' Parental AdvisorySLC' ' Parental Advisoryslc'
 ' Explicit Content' ' PARENTAL ADVISORY/EXPLICIT CONTENT' ' NO RATING'
 ' NOT IN ORACLE' ' Parental Guidance' ' Ages 10 & Up' ' Not in Oracle'
 ' MAX' ' Marvel Psr+' ' Ages 9+']


In [177]:
#UNIFICAMOS LAS CATEGORÍAS DE RATING
df_raw["Rating_clean"] = df_raw["Rating"].astype(str).str.lower().str.strip()
rating_map = {
    "none": "No Rated",
    "no rating": "No Rated",
    "not in oracle": "No Rated",
    "t+": "T+",
    "rated t+": "T+",
    "rated t": "T",
    "t": "T",
    "all ages": "All Ages",
    "a": "A",
    "rated a": "A",
    "parental advisory": "Parental Advisory",
    "parental advisory/explicit content": "Parental Advisory",
    "parental advisoryslc": "Parental Advisory",
    "explicit content": "Explicit Content",
    "marvel psr": "Marvel PSR",
    "marvel psr+": "Marvel PSR+",
    "max": "Max",
    "mature": "Mature",
    "parental guidance": "Parental Advisory",
    "parental supervision": "Parental Advisory",
    "ages 10 & up": "All Ages",
    "ages 9+": "All Ages",
    # puedes agregar más variantes si aparecen
}

df_raw["Rating"] = df_raw["Rating_clean"].replace(rating_map)

df_raw.drop(columns=["Rating_clean"], inplace=True)

# Ver cuántos valores únicos quedan
print(df_raw["Rating"].value_counts())




Rating
No Rated             22502
T+                    4410
T                     3057
Parental Advisory     1613
All Ages              1199
Marvel PSR            1060
A                      903
Explicit Content       149
Mature                  44
Max                     34
Marvel PSR+             21
Name: count, dtype: int64


In [178]:
#DUPLICADOS
num_duplicates = df_raw.duplicated().sum()
print(f"Hay {num_duplicates} registros duplicados")


Hay 769 registros duplicados


In [ ]:
#ELIMINAMOS LOS DUPLICADOS
df_raw = df_raw.drop_duplicates()

